In [1]:
# aleph

In [ ]:
import uproot
import numpy as np
import matplotlib.pyplot as plt

# -------------------------
# Load ROOT file
# -------------------------
file = uproot.open("QQB-3.root")
tree = file["events"]

# -------------------------
# Load branches (p is common)
# -------------------------
p_all = tree["pfcand_p"].array(library="np")

pads_all = {
    "ch": tree["pfcand_dEdx_ch_value_good"].array(library="np"),
    "el": tree["pfcand_dEdx_el_value_good"].array(library="np"),
    "mu": tree["pfcand_dEdx_mu_value_good"].array(library="np"),
    "ph": tree["pfcand_dEdx_ph_value_good"].array(library="np"),
    "nh": tree["pfcand_dEdx_nh_value_good"].array(library="np"),
}

wires_all = {
    "ch": tree["pfcand_dEdx_ch_wire_value_good"].array(library="np"),
    "el": tree["pfcand_dEdx_el_wire_value_good"].array(library="np"),
    "mu": tree["pfcand_dEdx_mu_wire_value_good"].array(library="np"),
    "ph": tree["pfcand_dEdx_ph_wire_value_good"].array(library="np"),
    "nh": tree["pfcand_dEdx_nh_wire_value_good"].array(library="np"),
}

# -------------------------
# Flatten nested RVecs
# -------------------------
data = {}
for label in ["ch", "el", "mu", "ph", "nh"]:
    p_flat, pads_flat, wires_flat = [], [], []

    for ev_idx in range(len(p_all)):
        p_event = p_all[ev_idx]
        pads_event = pads_all[label][ev_idx]
        wires_event = wires_all[label][ev_idx]

        for jet_idx in range(len(pads_event)):
            pads_jet = pads_event[jet_idx]
            wires_jet = wires_event[jet_idx]
            p_jet = p_event[jet_idx]

            for const_idx in range(len(pads_jet)):
                p_flat.append(float(p_jet[const_idx]))
                pads_flat.append(float(pads_jet[const_idx]))
                wires_flat.append(float(wires_jet[const_idx]))

    data[label] = {
        "p": np.array(p_flat),
        "pads": np.array(pads_flat),
        "wires": np.array(wires_flat)
    }

# -------------------------
# Colors for Pads and Wires
# -------------------------
colors = {
    "pads": {"ch": "white", "el": "yellow", "mu": "orange", "ph": "cyan", "nh": "magenta"},
    "wires": {"ch": "cyan", "el": "lime", "mu": "magenta", "ph": "orange", "nh": "white"}
}

# -------------------------
# Individual 2D scatter plots (dark background)
# -------------------------
plt.style.use('dark_background')
for label, d in data.items():
    # Pads
    plt.figure(figsize=(8,6), dpi=300)
    plt.gcf().set_facecolor('#000000')
    plt.scatter(d["p"], d["pads"], s=1, color=colors["pads"][label], alpha=0.5, label=f"{label.upper()} Pads")
    plt.xlabel("p [GeV]", color='white')
    plt.ylabel("dE/dx Pads [arb. units]", color='white')
    plt.xlim(0, 10)
    plt.ylim(0, 5)
    plt.title(f"dE/dx Pads vs Momentum ({label})", color='white')
    plt.legend(markerscale=5, fontsize=10, facecolor='#000000', edgecolor='white', labelcolor='white')
    plt.tight_layout()
    plt.savefig(f"dEdxPads_vs_p_{label}_dark.png")
    plt.close()

    # Wires
    plt.figure(figsize=(8,6), dpi=300)
    plt.gcf().set_facecolor('#000000')
    plt.scatter(d["p"], d["wires"], s=1, color=colors["wires"][label], alpha=0.5, label=f"{label.upper()} Wires")
    plt.xlabel("p [GeV]", color='white')
    plt.ylabel("dE/dx Wires [arb. units]", color='white')
    plt.xlim(0, 10)
    plt.ylim(0, 5)
    plt.title(f"dE/dx Wires vs Momentum ({label})", color='white')
    plt.legend(markerscale=5, fontsize=10, facecolor='#000000', edgecolor='white', labelcolor='white')
    plt.tight_layout()
    plt.savefig(f"dEdxWires_vs_p_{label}_dark.png")
    plt.close()

# -------------------------
# Combined overlay scatter plot (dark background)
# -------------------------
plt.figure(figsize=(10,7), dpi=300)
plt.gcf().set_facecolor('#000000')

# Plot Pads
for label in ["ch", "el", "mu", "ph", "nh"]:
    d = data[label]
    plt.scatter(d["p"], d["pads"], s=1, color=colors["pads"][label], alpha=0.5, label=f"{label.upper()} Pads")

# Plot Wires
for label in ["ch", "el", "mu", "ph", "nh"]:
    d = data[label]
    plt.scatter(d["p"], d["wires"], s=1, color=colors["wires"][label], alpha=0.5, label=f"{label.upper()} Wires")

plt.xlabel("p [GeV]", color='white')
plt.ylabel("dE/dx [arb. units]", color='white')
plt.xlim(0, 10)
plt.ylim(0, 5)
plt.title("Combined dE/dx Pads + Wires vs Momentum (all particles)", color='white')
plt.legend(markerscale=5, fontsize=10, facecolor='#000000', edgecolor='white', labelcolor='white')
plt.tight_layout()
plt.savefig("dEdxPadsWires_allParticles_dark.png")
plt.show()
